In [47]:
# 用户相关特征，14个，从训练集1、训练集2和测试集的特征区间提取：
#         count_merchant # 用户消费商户数量
#         user_avg_distance # 所有使用优惠券消费的商户与用户的平均距离
#         user_min_distance # 所有使用优惠券消费的商户与用户的最小距离
#         user_max_distance # 所有使用优惠券消费的商户与用户的最大距离
#         user_median_distance # 所有使用优惠券消费的商户与用户的中位距离

#         buy_use_coupon # 每个用户使用优惠券消费次数
#         buy_total # 用户消费次数
#         coupon_received # 用户领取优惠券次数

#         avg_user_date_datereceived_gap # 用户从领取优惠券到消费的平均时间间隔
#         min_user_date_datereceived_gap # 用户从领取优惠券到消费的最小时间间隔
#         max_user_date_datereceived_gap # 用户从领取优惠券到消费的最大时间间隔

#         user_coupon_transfer_rate = buy_use_coupon/coupon_received # 用户优惠券转化为实际消费比例
#         buy_use_coupon_rate = buy_use_coupon/buy_total # 用户使用优惠券消费占总消费的比例
#         user_date_datereceived_gap # 接受到优惠券的日期和使用之间的间隔

In [48]:
import pandas as pd

off_train = pd.read_csv('data/ccf_offline_stage1_train.csv',parse_dates=["Date"],header=0)
off_train.columns = ['user_id','merchant_id','coupon_id','discount_rate','distance','date_received','date']

off_test = pd.read_csv('data/ccf_offline_stage1_test_revised.csv',header=0)
off_test.columns = ['user_id','merchant_id','coupon_id','discount_rate','distance','date_received']

on_train = pd.read_csv('data/ccf_online_stage1_train.csv',parse_dates=["Date"],header=0)
on_train.columns = ['user_id','merchant_id','action','coupon_id','discount_rate','date_received','date']

# 【dataset3】20160701~20160731 (113640)   【feature3】20160315~20160630  （测试集）
dataset3 = off_test.dropna(how='any')
feature3 = off_train[((off_train.date >= '20160315') & (off_train.date <= '20160630'))
                   | ((off_train.date == 'null') & (off_train.date_received >= '20160315') 
                      &  (off_train.date_received <= '20160630'))
                   ].dropna(how='any')
# 【dataset2】20160515~20160615 (258446)   【feature2】20160201~20160514  （训练集2）
dataset2 = off_train[((off_train.date_received >= '20160515') & (off_train.date_received <= '20160615'))].dropna(how='any')
feature2 = off_train[((off_train.date >= '20160201') & (off_train.date <= '20160514'))
                   | ((off_train.date == 'null') & (off_train.date_received >= '20160201') 
                      &  (off_train.date_received <= '20160514'))
                   ].dropna(how='any')
# 【dataset1】20160414~20160514 (138303)   【feature1】20160101~20160413  （训练集1）
dataset1 = off_train[((off_train.date_received >= '20160414') & (off_train.date_received <= '20160514'))].dropna(how='any')
feature1 = off_train[((off_train.date >= '20160101') & (off_train.date <= '20160413'))
                   | ((off_train.date == 'null') & (off_train.date_received >= '20160101') 
                      &  (off_train.date_received <= '20160413'))
                   ].dropna(how='any')

In [49]:
from datetime import date
import numpy as np

In [50]:
def get_user_date_datereceived_gap(s):
    s = s.split(':')
    used = s[0]
    received = s[1]
    return ((date(int(used[0:4]),int(used[4:6]),int(used[6:8]))) - 
            (date(int(received[0:4]),int(received[4:6]),int(received[6:8])))).days

In [51]:
def userRelationFeature(feature,file_name):
    user3 = feature[['user_id', 'merchant_id', 'coupon_id', 'discount_rate', 'distance', 'date_received', 'date']]
    user3.ix[:5]

    t = user3[['user_id']]
    t.drop_duplicates(inplace=True)

    # count_merchant # 用户消费商户数量
    t1 = user3[user3.date != 'null'][['user_id','merchant_id']]
    t1.merchant_id = 1
    t1 = t1.groupby('user_id').agg('sum').reset_index()
    t1.rename(columns={'merchant_id': 'count_merchant'}, inplace=True)
    t1.ix[:5]

    t2 = user3[(user3.date != 'null') & (user3.coupon_id != 'null')][['user_id', 'distance']]
    t2.replace('null', -1, inplace=True)
    t2.distance = t2.distance.astype('int')
    t2.replace(-1, np.nan, inplace=True)
    t2.ix[:5]
    # user_mean_distance # 所有使用优惠券消费的商户与用户的平均距离
    t3 = t2.groupby('user_id').agg('mean').reset_index()
    t3.rename(columns={'distance': 'user_mean_distance'}, inplace=True)
    t3.ix[:5]
    # user_min_distance # 所有使用优惠券消费的商户与用户的最小距离
    t4 = t2.groupby('user_id').agg('min').reset_index()
    t4.rename(columns={'distance': 'user_min_distance'}, inplace=True)
    t4.ix[:5]
    # user_max_distance # 所有使用优惠券消费的商户与用户的最大距离
    t5 = t2.groupby('user_id').agg('max').reset_index()
    t5.rename(columns={'distance': 'user_max_distance'}, inplace=True)
    t5.ix[:5]
    # user_median_distance # 所有使用优惠券消费的商户与用户的中位距离
    t6 = t2.groupby('user_id').agg('median').reset_index()
    t6.rename(columns={'distance': 'user_median_distance'}, inplace=True)
    t6.ix[:5]

    # coupon_received # 用户领取优惠券次数
    t7 = user3[user3.coupon_id != 'null'][['user_id']]
    t7['coupon_received'] = 1
    t7 = t7.groupby('user_id').agg('sum').reset_index()
    t7.ix[:5]

    # buy_total # 用户消费次数
    t8 = user3[(user3.date != 'null')][['user_id']]
    t8['buy_total'] = 1
    t8 = t8.groupby('user_id').agg('sum').reset_index()
    t8.ix[:5]

    # buy_use_coupon # 每个用户使用优惠券消费次数
    t9 = user3[(user3.coupon_id != 'null')&(user3.date != 'null')][['user_id']]
    t9['buy_use_coupon'] = 1
    t9 = t9.groupby('user_id').agg('sum').reset_index()
    t9.ix[:5]

    t10 = user3[(user3.date != 'null')&(user3.date_received != 'null')][['user_id','date_received','date']]
    t10['user_date_datereceived_gap'] = t10.date + ":"+ t10.date_received
    t10.user_date_datereceived_gap = t10.user_date_datereceived_gap.apply(get_user_date_datereceived_gap)
    t10 = t10[['user_id','user_date_datereceived_gap']]
    t10.ix[:5]
    # avg_user_date_datereceived_gap # 用户从领取优惠券到消费的平均时间间隔
    t11 = t10.groupby('user_id').agg('mean').reset_index()
    t11.rename(columns={'user_date_datereceived_gap': 'avg_user_date_datereceived_gap'}, inplace=True)
    t11.ix[:5]
    # min_user_date_datereceived_gap # 用户从领取优惠券到消费的最小时间间隔
    t12 = t10.groupby('user_id').agg('min').reset_index()
    t12.rename(columns={'user_date_datereceived_gap': 'min_user_date_datereceived_gap'}, inplace=True)
    t12 = t12[['user_id','min_user_date_datereceived_gap']]
    t12.ix[:5]
    # max_user_date_datereceived_gap # 用户从领取优惠券到消费的最大时间间隔
    t13 = t10.groupby('user_id').agg('max').reset_index()
    t13.rename(columns={'user_date_datereceived_gap': 'max_user_date_datereceived_gap'}, inplace=True)
    t13 = t13[['user_id','max_user_date_datereceived_gap']]
    t13.ix[:5]

    # 合并数据
    user3_feature = pd.merge(t, t1, on='user_id', how='left')
    user3_feature = pd.merge(user3_feature, t3, on='user_id', how='left')
    user3_feature = pd.merge(user3_feature, t4, on='user_id', how='left')
    user3_feature = pd.merge(user3_feature, t5, on='user_id', how='left')
    user3_feature = pd.merge(user3_feature, t6, on='user_id', how='left')
    user3_feature = pd.merge(user3_feature, t7, on='user_id', how='left')
    user3_feature = pd.merge(user3_feature, t8, on='user_id', how='left')
    user3_feature = pd.merge(user3_feature, t9, on='user_id', how='left')
    user3_feature = pd.merge(user3_feature, t10, on='user_id', how='left')
    user3_feature = pd.merge(user3_feature, t11, on='user_id', how='left')
    user3_feature = pd.merge(user3_feature, t12, on='user_id', how='left')
    user3_feature = pd.merge(user3_feature, t13, on='user_id', how='left')

    user3_feature = user3_feature.replace(np.nan, 0)
    user3_feature.ix[:5]

    # user_coupon_transfer_rate = buy_use_coupon/coupon_received # 用户优惠券转化为实际消费比例
    user3_feature['user_coupon_transfer_rate'] = user3_feature.buy_use_coupon/user3_feature.coupon_received
    user3_feature = user3_feature.replace(np.nan, 0)
    user3_feature.ix[:5]
    # buy_use_coupon_rate = buy_use_coupon/buy_total # 用户使用优惠券消费占总消费的比例
    user3_feature['buy_use_coupon_rate'] = user3_feature.buy_use_coupon/user3_feature.buy_total
    user3_feature = user3_feature.replace(np.nan, 0)
    user3_feature.ix[:5]
    user3_feature.to_csv(file_name, index=None)
    return user3_feature

In [52]:
user3_feature = userRelationFeature(feature3,'data/user3_feature.csv')
user3_feature.ix[:5]

C:\Users\xhb_1\Anaconda3\lib\site-packages\pandas\util\decorators.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return func(*args, **kwargs)


,user_id,count_merchant,user_mean_distance,user_min_distance,user_max_distance,user_median_distance,coupon_received,buy_total,buy_use_coupon,user_date_datereceived_gap,avg_user_date_datereceived_gap,min_user_date_datereceived_gap,max_user_date_datereceived_gap,user_coupon_transfer_rate,buy_use_coupon_rate
0,1439408,2.0,0.0,0.0,0.0,0.0,4.0,2.0,1.0,28.0,28.0,28.0,28.0,0.25,0.5
1,1832624,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
2,2029232,2.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
3,2747744,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
4,196342,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
5,163606,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0


In [53]:
user2_feature = userRelationFeature(feature2,'data/user2_feature.csv')
user2_feature.ix[:5]

C:\Users\xhb_1\Anaconda3\lib\site-packages\pandas\util\decorators.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return func(*args, **kwargs)


,user_id,count_merchant,user_mean_distance,user_min_distance,user_max_distance,user_median_distance,coupon_received,buy_total,buy_use_coupon,user_date_datereceived_gap,avg_user_date_datereceived_gap,min_user_date_datereceived_gap,max_user_date_datereceived_gap,user_coupon_transfer_rate,buy_use_coupon_rate
0,1439408,1.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1832624,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,73611,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,163606,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,94107,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,253750,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
user1_feature = userRelationFeature(feature1,'data/user1_feature.csv')
user1_feature.ix[:5]

C:\Users\xhb_1\Anaconda3\lib\site-packages\pandas\util\decorators.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return func(*args, **kwargs)


,user_id,count_merchant,user_mean_distance,user_min_distance,user_max_distance,user_median_distance,coupon_received,buy_total,buy_use_coupon,user_date_datereceived_gap,avg_user_date_datereceived_gap,min_user_date_datereceived_gap,max_user_date_datereceived_gap,user_coupon_transfer_rate,buy_use_coupon_rate
0,1439408,1.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2029232,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2223968,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,73611,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3273056,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,94107,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
